In [1]:
import networkx as nx
import numpy as np
import torch
from datetime import datetime
import os
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.nn import GCNConv
import matplotlib.pyplot as plt

import pandas as pd
import torch_geometric.transforms as T

#from create_dataset import CreateDataset

from models import GNNModel

import sys
sys.path.append("../FastCover/")
from utils import *

In [2]:
PATH_SAVE_TRAINS_CHECKPOINTS = 'runs/checkpoints/'
PATH_SAVE_TRAINS = 'runs/'
PATH_TRAIN = '../FastCover/data/ER_graphs/train/'

num_features = 1
num_classes  = 2

threshold = 0.5

optimizer_name = "Adam"
lr = 1e-3
epochs = 20

SEED = 13

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

#layers = ["GCN", "GAT","GraphConv"]
layers = ["SAGE"]

Models = [GNNModel(c_in = 1, c_hidden = 100, c_out = 2, num_layers = 2, layer_name = layer_name, dp_rate=0.1) for 
         layer_name in layers]

In [3]:
ERInstances = [graph for graph in os.listdir(PATH_TRAIN+'pkl')]
graphs = []
for er in ERInstances:
    graph = igraph.Graph().Read_Pickle(PATH_TRAIN + 'pkl/'+er)
    graphs.append(graph.to_networkx())    

OptInstances = [graph for graph in os.listdir(PATH_TRAIN+'optimal')]
Solutions = []
for er in OptInstances:
    opt = []
    with open(PATH_TRAIN+'optimal/'+er) as f:
        for line in f.readlines():
            opt.append(int(line.replace("\n", "")))
    Solutions.append(opt)

In [4]:
Graphs_Train = Convert2DataSet(graphs, Solutions)
data = Graphs_Train[0]

In [85]:
S = np.argsort(-np.array(nx.degree(graphs[0])).T[1])[100:]
data = Convert2DataSet([graphs[0]], [S])[0]

In [86]:
np.array(Solutions[0])

array([ 13,  17,  24,  42,  46,  51,  55,  58,  63,  70,  72,  82,  83,
        86,  89, 113, 125, 143, 148, 172, 176, 179, 184, 190, 191, 194,
       204, 218, 225, 231, 233, 237, 239, 243, 245, 246, 258, 259, 262,
       266, 287, 293, 309, 316, 322, 336, 344, 346, 348, 354, 358, 359,
       360, 362, 373, 387, 391, 403, 404, 406, 410, 417, 422, 436, 453,
       454, 462, 473, 475, 477, 483, 490, 492, 498, 509, 518, 527, 543,
       547, 553, 557, 566, 568, 570, 575, 580, 593, 605, 623, 650, 657,
       671, 678, 683, 684, 685, 689, 691, 698, 703, 715, 732, 734, 738,
       748, 757, 775, 780, 782, 784, 786, 797, 801, 805, 806, 821, 822,
       829, 835, 852, 861, 863, 873, 882, 888, 896, 901, 903, 904, 907,
       916, 921, 929, 938, 943, 949, 951, 960, 962, 964, 965, 972, 973,
       975, 977, 979, 987, 999])

In [40]:
np.sort(np.argsort(-np.array(nx.degree(graphs[0])).T[1])[:148])

array([  0,   2,   7,  13,  17,  42,  46,  55,  58,  60,  63,  64,  67,
        68,  70,  75,  82,  86,  89, 108, 113, 160, 167, 173, 184, 188,
       190, 191, 194, 199, 200, 201, 204, 212, 218, 233, 234, 237, 239,
       244, 246, 254, 256, 259, 262, 263, 281, 283, 287, 301, 316, 319,
       322, 336, 344, 354, 357, 358, 359, 362, 390, 398, 402, 429, 430,
       432, 433, 453, 469, 473, 475, 483, 490, 492, 525, 526, 543, 547,
       553, 557, 566, 570, 578, 580, 582, 584, 593, 594, 601, 605, 609,
       619, 620, 631, 645, 650, 657, 663, 671, 684, 685, 697, 703, 711,
       732, 734, 742, 754, 757, 764, 768, 772, 775, 784, 787, 790, 797,
       801, 803, 806, 814, 824, 827, 835, 839, 840, 843, 849, 851, 852,
       859, 882, 888, 902, 903, 904, 916, 920, 929, 938, 943, 957, 960,
       972, 973, 974, 984, 999], dtype=int64)

In [94]:
import torch
import torch.nn.functional as F
from torch_geometric.nn import GCNConv

class GCN(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = GCNConv(data.num_node_features, 16)
        self.conv2 = GCNConv(16, data.num_classes)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index

        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = F.dropout(x, training=self.training)
        x = self.conv2(x, edge_index)

        return F.log_softmax(x, dim=1)

In [119]:
GAT = Models[0]

In [5]:
for i in range(len(Models)):
    print()
    print(f" ----- Model:{layers[i]} -----")
    for data in Graphs_Train:
        device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        model = Models[i].to(device)
        data = data.to(device)
        optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)

        model.train()
        for epoch in range(epochs):
            optimizer.zero_grad()
            #out = model(data)
            out = model(data.x, data.edge_index)
            loss = F.nll_loss(out, data.y)
            loss.backward()
            optimizer.step()
            
    dt_string = datetime.now().strftime("%m-%d_%H-%M")
    torch.save(model.state_dict(), f=f"{PATH_SAVE_TRAINS}{layers[i]}_seed_{SEED}_thr_{int(threshold*10)}_date_{dt_string}.pt")
    
    print(f"{layers[i]} saved in {PATH_SAVE_TRAINS}\n")


 ----- Model:SAGE -----
SAGE saved in runs/



In [161]:
for data in Graphs_Train[-1:]:
    model.eval()
    #pred = model(data).argmax(dim=1)
    pred = model(data.x, data.edge_index).argmax(dim=1)
    correct = (pred == data.y).sum()
    acc = int(correct) / 1000
    print(f'Accuracy: {acc:.4f}')
    print(torch.sum(pred))

Accuracy: 0.8690
tensor(158)


In [162]:
Top = np.sort(torch.topk(out.max(1)[0], 148)[1].detach().numpy())
TopDegree = np.sort(np.argsort(-np.array(nx.degree(graphs[-1])).T[1])[:148])
TopSol = np.where(data.y ==1)[0]

In [163]:
d = 0
s = 0
for t in Top:
    d += 1 if t in TopDegree else 0
    s += 1 if t in TopSol else 0
    

In [165]:
d

3

In [173]:
torch.exp(torch.log(torch.tensor([10])))

tensor([10.])

In [169]:
torch.tensor([1])

tensor([1])